In [88]:
import nltk
import numpy as np
import tokenize
from sklearn import preprocessing as prep
from nltk.tokenize import *
from nltk.corpus import stopwords as st
from string import punctuation

In [161]:
docs = []
posting_list = []     # [word, number of repitition, [docIDs]]

In [89]:
class Node:
    
    def __init__(self, data, next=None): 
        self.data = data
        self.next = next

In [90]:
class LinkedList:
    
    def __init__(self):
        self.head = None

    # insertion method for the linked list
    def insert(self, data):
        newNode = Node(data)
        if(self.head):
            current = self.head
            while(current.next):
                current = current.next
            current.next = newNode
        else:
            self.head = newNode
    
    # print method for the linked list
    def printLL(self):
        current = self.head
        while(current):
            print(current.data)
            current = current.next

In [91]:
# define a new class for documents

class Document:

    def __init__(self, ID) -> None:
        self.id = int(ID)                  # .I
        self.title = ""                    # .T
        self.abstract = ""                 # .W
        self.pub_date = 0                  # .B
        self.author = list()               # .A
        self.info = ""                     # .N
        self.ref = list()                  # .X

    def set_title(self, title):
        self.title = title
    
    def set_abstract(self, abstract):
        self.abstract += abstract

    def set_pubdate(self, pubdate):
        self.pubdate = int(pubdate)

    def set_author(self, author):
        self.author.append(author)

    def set_info(self, info):
        self.info += info

    def set_ref(self, ref):
        self.ref.append(ref)

    def get_id(self):
        return self.id

    def get_title(self):
        return self.title

    def get_abstract(self):
        return self.abstract

    def get_pubdate(self):
        return self.pub_date

    def get_author(self):
        return self.author

    def get_info(self):
        return self.info

    def get_ref(self):
        return self.ref
    

In [147]:
#read CISI.ALL file and fill lists


def read_from_file():

    # some flags for reading file properly
    t = 0
    a = 0
    w = 0
    x = 0
    n = 0
    b = 0
    
    with open("CISI.ALL", encoding = "latin-1") as all_files:
        current_doc = None
        for line in all_files.readlines():
            if line.startswith(".I"):
                t = 0
                a = 0
                w = 0
                x = 0
                n = 0
                b = 0
                id = line[3]
                current_doc = Document(id)
                docs.append(current_doc)
            elif line.startswith(".T"):
                t = 1
                a = 0
                w = 0
                x = 0
                n = 0
                b = 0
            elif line.startswith(".W"):
                t = 0
                a = 0
                w = 1
                x = 0
                n = 0
                b = 0
            elif line.startswith(".B"):
                t = 0
                a = 0
                w = 0
                x = 0
                n = 0
                b = 1
            elif line.startswith(".N"):
                t = 0
                a = 0
                w = 0
                x = 0
                n = 1
                b = 0
            elif line.startswith(".X"):
                t = 0
                a = 0
                w = 0
                x = 1
                n = 0
                b = 0
            elif line.startswith(".A"):
                t = 0
                a = 1
                w = 0
                x = 0
                n = 0
                b = 0
            elif t == 1:
                # title
                current_doc.set_title(line.lower())
                t = 0
            elif a == 1:
                # author
                current_doc.set_author(line.lower())
            elif b == 1:
                # publication date
                current_doc.set_pubdate(line.lower())
            elif w == 1:
                # abstract
                current_doc.set_abstract(line.lower())
            elif x == 1:
                # references
                current_doc.set_ref(line.lower())
            elif n == 1:
                # info
                current_doc.set_info(line.lower())
            else:
                continue

            
    all_files.close()


In [204]:
# building posting list

def preprocessing(): 
    tokens = []
    normalized = []
    stopwords = st.words("english")
    
    # tokenization
    for doc in docs:
        abstract = doc.get_abstract()
        tokenizer = TweetTokenizer()
        tokens = tokenizer.tokenize(abstract)
        
        # remove stopwords
        for token in tokens:
            if (token not in stopwords) & (token not in punctuation) & (any(token not in row for row in posting_list)):
                linked_list = LinkedList()
                linked_list.insert(int(doc.get_id()))
                posting_list.append([token, 1, linked_list])
            elif (any(token in row for row in posting_list)):
                row = posting_list.index(token)
                posting_list[row][1] += 1
                linked_list = posting_list[row][2]
                linked_list.insert(int(doc.get_id()))
                posting_list[row][2] = linked_list
            
            
    # normalization
    posting_list[:,0] = prep.normalize([posting_list[:,0]])


In [203]:
# test

read_from_file()
preprocessing()
# print(posting_list)

KeyboardInterrupt: 